In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from load_dotenv import load_dotenv

In [ ]:
load_dotenv()
DOTA_DIR = Path(os.getenv("DOTA_DIR"))
detection_dir = DOTA_DIR / "predictions"
CSV_PATH = detection_dir / "detections_zone2.csv"
CONF_TH  = 0.05  # seuil de confiance principal

In [ ]:
CLASS_MAP = {
    9: "large vehicle",
    10: "small vehicle"
}

In [ ]:
df = pd.read_csv(CSV_PATH)
# Nettoyage minimal / types
cols = ["x1", "y1", "x2", "y2", "conf", "class"]
df = df[cols].copy()
df[["x1","y1","x2","y2","conf"]] = df[["x1","y1","x2","y2","conf"]].apply(pd.to_numeric, errors="coerce")
df["class"] = pd.to_numeric(df["class"], errors="coerce").astype("Int64")
df = df.dropna(subset=["x1","y1","x2","y2","conf","class"])
df["class_name"] = df["class"].map(CLASS_MAP)

In [ ]:
# 1. Statistiques de base
print("=== Statistiques générales ===")
print(f"Nombre total de détections: {len(df)}")
print(f"Nombre de classes uniques: {df['class'].nunique()}")
print("\nRépartition par classe:")
print(df['class_name'].value_counts())

In [ ]:
# 2. Statistiques sur les coordonnées des boxes
print("\n=== Statistiques sur les coordonnées ===")
print(df[['x1', 'y1', 'x2', 'y2']].describe())

# 3. Statistiques sur la confiance
print("\n=== Statistiques sur la confiance ===")
print(df['conf'].describe())

In [ ]:
# 4. Calcul de la taille des boxes (largeur et hauteur)
df['width'] = df['x2'] - df['x1']
df['height'] = df['y2'] - df['y1']
df['area'] = df['width'] * df['height']

print("\n=== Statistiques sur la taille des boxes ===")
print(df[['width', 'height', 'area']].describe())

In [ ]:
# 5. Visualisations
plt.figure(figsize=(15, 10))

# Histogramme des confiances
plt.subplot(2, 2, 1)
sns.histplot(df['conf'], bins=30, kde=True)
plt.title('Distribution des scores de confiance')
plt.xlabel('Confiance')

# Boxplot des tailles par classe
plt.subplot(2, 2, 2)
sns.boxplot(x='class_name', y='area', data=df)
plt.title('Distribution de la taille des boxes par classe')
plt.yscale('log')  # Échelle logarithmique car les tailles peuvent varier beaucoup

# Scatter plot des positions (échantillon pour éviter la surcharge)
plt.subplot(2, 2, 3)
sample_df = df.sample(min(1000, len(df)))  # Prendre max 1000 points
sns.scatterplot(x='x1', y='y1', hue='class', data=sample_df, alpha=0.6)
plt.title('Position des détections (échantillon)')
plt.gca().invert_yaxis()  # Les images ont souvent y=0 en haut

# Distribution des rapports largeur/hauteur
plt.subplot(2, 2, 4)
df['aspect_ratio'] = df['width'] / df['height']
sns.histplot(df['aspect_ratio'], bins=30)
plt.title('Distribution du rapport largeur/hauteur')
plt.xlim(0, 5)  # Limiter pour éviter les valeurs extrêmes

plt.tight_layout()
plt.show()

In [ ]:
# 6. Statistiques par classe
print("\n=== Statistiques par classe ===")
for cls in df['class_name'].unique():
    cls_df = df[df['class_name'] == cls]
    print(f"\nClasse {cls}:")
    print(f"- Nombre de détections: {len(cls_df)}")
    print(f"- Confiance moyenne: {cls_df['conf'].mean():.3f}")
    print(f"- Taille moyenne (LxH): {cls_df['width'].mean():.1f}x{cls_df['height'].mean():.1f}")
    print(f"- Aire moyenne: {cls_df['area'].mean():.1f}")

# 7. Détection des valeurs aberrantes
print("\n=== Valeurs aberrantes potentielles ===")
# Boxes très grandes (peut-être des erreurs de détection)
large_boxes = df[df['area'] > df['area'].quantile(0.99)]
print(f"Boxes très grandes (>99ème percentile): {len(large_boxes)}")
print(large_boxes[['x1', 'y1', 'x2', 'y2', 'area', 'conf', 'class']].head())

# Boxes avec très faible confiance
low_conf = df[df['conf'] < 0.1]
print(f"\nDétections à très faible confiance (<0.1): {len(low_conf)} ({len(low_conf)/len(df)*100:.1f}%)")